In [ ]:
!pip install textblob
!pip install tweepy
!pip install pycountry
!pip install wordcloud
!pip install langdetect
!pip install nltk
!pip install geopy
!pip install google_trans_new

In [ ]:
from textblob import TextBlob
import sys
import json
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('vader_lexicon')
nltk.download('stopwords')
import pycountry
import re
import string
from google_trans_new import google_translator
import time  
from geopy.geocoders import Nominatim
import warnings
warnings.simplefilter('ignore')
  

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
geolocator = Nominatim(user_agent="locator")

In [ ]:
d = pd.read_json('nazioni.json', lines=True)
d.head()

In [ ]:
df=d[d['user_location'].notna()] 
dfna=d[d['user_location'].isna()]

In [ ]:
df['user_location']=df['user_location'].str.lower()
df.head()

In [ ]:
#Separo il dataset con le user_location non nulle contenti luoghi geografici geolocalizzabili
df_clean=df[~df['user_location'].str.contains('global|earth|world|europe|asia|oceania|planet|somewhere|nowhere|everywhere')].reset_index(drop=True)
df_toclean=df[df['user_location'].str.contains('global|earth|world|europe|asia|oceania|planet|somewhere|nowhere|everywhere')].reset_index(drop=True)

In [ ]:
df_toclean=df_toclean.append(dfna)

In [ ]:
#Funzione per geolocalizzare
df_clean['country']=None
for i in range(0,len(df_clean['user_location'])):
    try:
        g=geolocator.geocode(df_clean['user_location'][i])
        time.sleep(1)
        if g is not None:
            df_clean['country'][i]=g[0].split(", ")[-1]
        else:
            df_clean['country'][i]=None
    except:
        pass
    if i%60==0:
        print(int(i/60))
    else:
        pass

In [ ]:
dfprovv=df_clean.copy(deep=True)

In [ ]:
dfprovv1=dfprovv[dfprovv['country'].notna()]
dfprovv2=dfprovv[dfprovv['country'].isna()]

In [ ]:
df_toclean=df_toclean.append(dfprovv2)

In [ ]:
df_toclean.reset_index(drop=True, inplace=True)

In [ ]:
data=dfprovv[dfprovv['country'].notna()]

In [ ]:
df_toclean.groupby('lingua').count().sort_values(by='username', ascending=False)

In [ ]:
data.groupby('lingua').count().sort_values(by='username', ascending=False)

In [ ]:
data.reset_index(drop=True, inplace=True)

In [ ]:
data.groupby('country').count().sort_values(by='lingua', ascending=False)['username']

In [ ]:
dfexport=pd.DataFrame(data.groupby('country').count().sort_values(by='lingua', ascending=False)['username'])

In [ ]:
dfexport.rename(columns={'username':'count'}, inplace=True)

In [ ]:
dfexport.reset_index(inplace=True)

In [ ]:
#Traduco i country in lingua inglese
lista_paesi=list(dfexport['country'])
paesi_en={}
translator = google_translator() 
for i in lista_paesi:
    paesi_en[i]=translator.translate(i.split('/')[0],lang_tgt='en')

In [ ]:
data['country_en']=data['country'].map(paesi_en)

In [ ]:
data.head(10)

In [ ]:
country_dict={'Argentina ':'Argentina',
 'Australia ':'Australia',
 'Austria ':'Austria',
 'Belgium ':'Belgium',
 'Brazil ':'Brazil',
 'Canada ':'Canada',
 'chili ':'Chile',
 'Colombia ':'Colombia',
 'Denmark. ':'Denmark',
 'Finland ':'Finland',
 'France ':'France',
 'Germany ':'Germany',
 'Hellas ':'Greece',
 'Iceland ':'Iceland',
 'India ':'India',
 'Indonesia ':'Indonesia',
 'Ireland ':'Ireland',
 'Italy ':'Italy',
 'Japan ':'Japan',
 'Kenya ':'Kenya',
 'Luxembourg ':'Luxembourg',
 'Mexico ':'Mexico',
 'The Netherlands ':'Netherlands',
 'New Zealand ':'New Zealand',
 'Norway ':'Norway',
 'Pakistan ':'Pakistan',
 'Poland ':'Poland',
 'Portugal ':'Portugal',
 'South Africa ':'South Africa',
 'Spain ':'Spain',
 'Sweden ':'Sweden',
 'Switzerland ':'Switzerland',
 'Thailand ':'Thailand',
 'Uganda ':'Uganda',
 'United Kingdom ':'United Kingdom',
 'United States ':'United States'}

In [ ]:
data['country_norm']=data['country_en'].map(country_dict)

In [ ]:
li=country_dict.values()

In [ ]:
data=data[data['country_norm'].isin(li)]

In [ ]:
pd.unique(data['country_norm'])

In [ ]:
data.head()

In [ ]:
data = data.rename(columns={'country_norm':'Country'})[['username','testo', 'lingua','Country']]
data.head()

In [ ]:
data.reset_index(inplace=True)

FASE DI SENTIMENT

Traduco il testo in lingua inglese creando una colonna da associare al testo originale, alla lingua e al country.

In [ ]:
testo=[]
for i in range(0, len(data)):
    if data['lingua'][i]=='en':
        testo.append(data['testo'][i])
    else:
        translation = translator.translate(data['testo'][i], lang_tgt='en')  
        testo.append(translation)
    time.sleep(1)

In [ ]:
dataTweet = pd.DataFrame(testo)
dataTweet = dataTweet.rename(columns={0: 'testo'})
dataTweet['originale']=data['testo']
dataTweet['lingua']=data['lingua']
dataTweet['country']=data['Country']

Pulizia del testo tradotto da retweet e simboli particolari.

In [ ]:
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
dataTweet["testo"] = dataTweet.testo.map(remove_rt).map(rt)
dataTweet["testo"] = dataTweet.testo.str.lower()

In [ ]:
# Pulizia Manuale degli hashtag 
final=[]
for i in range(0, len(dataTweet)):
    s = dataTweet['testo'][i]
    replaced = re.sub("diadelatierra", 'world earth day', s)
    replaced = re.sub('earthday', "earth day", replaced)
    replaced = re.sub('ii', "i", replaced)
    replaced = re.sub('climateaction', "climate action", replaced)
    replaced = re.sub('isnt', "is", replaced)
    replaced = re.sub('climatestrikesback', "climate strike is back", replaced)
    replaced = re.sub('climatestrike', "climate strike", replaced)
    replaced = re.sub('plasticbagfreeday', "plastic bag free day", replaced)
    replaced = re.sub('worldenvironmentday', "world environment day", replaced)
    replaced = re.sub('fridaysforfuture', "fridays for future", replaced)
    replaced = re.sub('climatechange', "climate change", replaced)
    replaced = re.sub('savetheplanet', "save the planet", replaced)
    replaced = re.sub('worldearthday', 'world earth day', replaced)
    replaced = re.sub('worldoceanday', 'world ocean day', replaced)
    replaced = re.sub('changementclimatique', 'climate change', replaced)
    replaced = re.sub('cambiamentoclimatico', 'climate change', replaced)
    replaced = re.sub('cambioclimatico', 'climate change', replaced)
    replaced = re.sub('plasticfree', 'plastic free', replaced)
    replaced = re.sub('ecofriendly', 'eco friendly', replaced)
    replaced = re.sub('globalwarming', 'global warming', replaced)
    replaced = re.sub('jourdelaterre', 'world earth day', replaced)
    replaced = re.sub('diamundialdelmedioambiente', 'world earth day', replaced)
    replaced = re.sub('diamundialdelmedioambient', 'world earth day', replaced)
    replaced = re.sub('earthdayeveryday', 'earth day every day', replaced)
    replaced = re.sub('naturephotography', 'nature photography', replaced)
    replaced = re.sub('cambioclimatic', 'climate change', replaced)
    replaced = re.sub('meiobiente', 'my environment', replaced)
    replaced = re.sub('domelatierra', 'world earth day', replaced)
    replaced = re.sub('adelatierra', 'world earth day', replaced)
    replaced = re.sub('cinemastrikesback', 'cinema strike is back', replaced)
    replaced = re.sub('twitternaturecommunity', 'twitter nature community', replaced)
    replaced = re.sub('birdwatching', 'bird watching', replaced)
    replaced = re.sub('zerowaste', 'zero waste', replaced)
    replaced = re.sub('trees', 'tree', replaced)
    replaced = re.sub('giarnatamentialedellaterra', 'world earth day', replaced)
    replaced = re.sub('generationrestoration', 'world earth day', replaced)
    replaced = re.sub('tweetforatree', 'tweet for a tree', replaced)
    replaced = re.sub('plantforlakshadweep', 'plant for lakshadweep', replaced)
    replaced = re.sub('adelatierra', 'world earth day', replaced)
    replaced = re.sub('climatecrisis', 'climate crisis', replaced)

    final.append(replaced)

dataTweet['testo']=final

Calcolo degli indici relativi alla Sentiment Analysis tra i quali la polarità.

In [ ]:
dataTweet[['polarity', 'subjectivity']] = dataTweet['testo'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in dataTweet['testo'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        dataTweet.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        dataTweet.loc[index, 'sentiment'] = "positive"
    else:
        dataTweet.loc[index, 'sentiment'] = "neutral"
    dataTweet.loc[index, 'neg'] = neg
    dataTweet.loc[index, 'neu'] = neu
    dataTweet.loc[index, 'pos'] = pos
    dataTweet.loc[index, 'compound'] = comp

Creo una varaibile categorica che assegna il sentimento al tweet sulla base della polarity.

In [ ]:
pol=[]
for i in range(0, len(dataTweet['polarity'])):
    if dataTweet['polarity'][i] < 0:
        pol.append("negative")
    elif dataTweet['polarity'][i] > 0:
        pol.append("positive")
    else:
        pol.append("neutral")

dataTweet['p_sent']=pol

Creo una lista per ogni sentimento contenente i tweet associati.

In [ ]:
tw_list_negative = dataTweet[dataTweet["sentiment"]=="negative"]
tw_list_positive = dataTweet[dataTweet["sentiment"]=="positive"]
tw_list_neutral = dataTweet[dataTweet["sentiment"]=="neutral"]

In [ ]:
def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [ ]:
count_values_in_column(dataTweet,"sentiment")

In [ ]:
count_values_in_column(dataTweet,"p_sent")

#Inizio fase di text analysis

In [ ]:
# Lunghezza Tweet e conteggio parole
dataTweet['text_len'] = dataTweet['testo'].astype(str).apply(len)
dataTweet['text_word_count'] = dataTweet['testo'].apply(lambda x: len(str(x).split()))
dataTweet
round(dataTweet.text_len.mean(),2)
round(pd.DataFrame(dataTweet.groupby("sentiment").text_len.mean()),2)

In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

dataTweet['punct'] = dataTweet['testo'].apply(lambda x: remove_punct(x))
dataTweet.head()

In [ ]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

dataTweet['tokenized'] = dataTweet['punct'].apply(lambda x: tokenization(x.lower()))

In [ ]:
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
dataTweet['nonstop'] = dataTweet['tokenized'].apply(lambda x: remove_stopwords(x))

In [ ]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

dataTweet['stemmed'] = dataTweet['nonstop'].apply(lambda x: stemming(x))

In [ ]:
stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

In [ ]:
def clean_text(text):
    text = [word for word in text if word not in stopword]
    return text

In [ ]:
def get_top_n_gram(corpus,ngram_range,n=None):
    vec = CountVectorizer(ngram_range=ngram_range,stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    dizi={}
    for word, idx in vec.vocabulary_.items():
        dizi[word]=sum_words[0, idx]
    dizionario = {'words': list(dizi.keys()), 'freq': list(dizi.values())}
    df3=pd.DataFrame.from_dict(dizionario)
    df3=df3.sort_values(by='freq', ascending=False)
    df_final=df3.head(n)
    return df_final

In [ ]:
def create_wordcloud(text, per):
    stopwords = nltk.corpus.stopwords.words('english')
    wc = WordCloud(background_color="white",
                  max_words=200,
                  stopwords=stopwords,
                  repeat=False)
    wc.generate(str(text))
    wc.to_file(per)
    print("Word Cloud Saved Successfully")
    display(Image.open(per))

In [ ]:
def textanalysis(paese, pp):
    dataset=dataTweet[dataTweet['country']==paese]

    countVectorizer = CountVectorizer(analyzer=clean_text) 
    countVector = countVectorizer.fit_transform(dataset['nonstop'])
    df4= pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
    count = pd.DataFrame(df4.sum())
    countdf = count.sort_values(0,ascending=False).head(20)
    words=countdf.reset_index()
    n2_bigrams = get_top_n_gram(dataset['testo'],(2,2),20)
    n3_trigrams = get_top_n_gram(dataset['testo'],(3,3),20)
    create_wordcloud(text=dataset['nonstop'], per=pp)

    final= pd.concat([words,n2_bigrams,n3_trigrams], axis=1)

    return final


In [ ]:
# costruzione df per viz
dfwordcloud=pd.DataFrame(columns=["words", "freq", "country"])
df_n_gram=pd.DataFrame(columns=["bigram", "freq_2", "trigram", "freq_3", "country"])

In [ ]:
#Creo due dataset contenenti per nazione le parole e i bigrammi/tigrammi più frequenti
for i in np.unique(dataTweet['country']):
    data3 = dataTweet[dataTweet['country']==i].reset_index()
    print(i)
    prova=[]
    for riga in range(0, len(data3)):
        for word in data3['nonstop'][riga]:
            if len(word)>1:
                prova.append(word)
  
    dfprova=pd.DataFrame(prova)
    dfprova = dfprova.rename(columns={0: 'words'})
    dfprova['freq']=1
    freq_words=dfprova.groupby("words").count().sort_values(by='freq', ascending=False)
    final=freq_words.head(300).reset_index()
    final['country']=i
    n2_bigrams = get_top_n_gram(data3['testo'],(2,2),50)
    n3_trigrams = get_top_n_gram(data3['testo'],(3,3),50)

    f=pd.DataFrame()
    f['bigram']=n2_bigrams.reset_index()['words']
    f['freq_2']=n2_bigrams.reset_index()['freq']
    f['trigram']=n3_trigrams.reset_index()['words']
    f['freq_3']=n3_trigrams.reset_index()['freq']
    f['country']=i

    dfwordcloud=dfwordcloud.append(final)
    df_n_gram=df_n_gram.append(f)

Salvo i due dataset per la parte di data viz; sono commentati per facilitare l'esecuzione del codice.

In [ ]:
#dfwordcloud.to_csv("/content/drive/MyDrive/Magistrale/DM&DV Project/Dati/df_wordcloud.csv")

In [ ]:
#df_n_gram.to_csv("/content/drive/MyDrive/Magistrale/DM&DV Project/Dati/df_n_gram.csv")

# Dataset aggregato

In [ ]:
neg=pd.DataFrame(dataTweet.groupby("country").neg.mean())
pos=pd.DataFrame(dataTweet.groupby("country").pos.mean())
neu=pd.DataFrame(dataTweet.groupby("country").neu.mean())
polarity=pd.DataFrame(dataTweet.groupby("country").polarity.mean())

In [ ]:
neg_std=pd.DataFrame(dataTweet.groupby("country").neg.std()).rename(columns={'neg':'neg_std'})
pos_std=pd.DataFrame(dataTweet.groupby("country").pos.std()).rename(columns={'pos':'pos_std'})
neu_std=pd.DataFrame(dataTweet.groupby("country").neu.std()).rename(columns={'neu':'neu_std'})
polarity_std=pd.DataFrame(dataTweet.groupby("country").polarity.std()).rename(columns={'polarity':'polarity_std'})

In [ ]:
result = pd.concat([neg, neg_std, pos, pos_std, neu, neu_std, polarity, polarity_std], axis=1)
result.head()

In [ ]:
result.reset_index(inplace=True)

#Importazione e pulizia e dati indicatori

Il dataset è stato recuperato mediante una query sul sito della WorldBank.

In [ ]:
data2 = pd.read_csv('Socio_economic_inds.csv')
data2.head()

Seleziono le colonne di interesse e creo una lista con i nomi delle stesse al fine di rimuovere le righe presentanti valori mancanti per tali colonne.

In [ ]:
data2=data2.drop('Series Code', axis=1)

In [ ]:
cols=data2.columns

In [ ]:
df = data2[cols].replace({'..':np.nan})

In [ ]:
df.dropna(subset=cols[range(3,len(cols))], how='all', inplace=True)

Creo due liste che raccoglieranno i valori più recenti per ogni indicatore di ogni stato andando a salvare anche l'anno relativo. Le accodo poi al dataset.

In [ ]:
li=['None']*len(df)
year =['None']*len(df)

In [ ]:
for i in range(0,len(df)):
    j=13
    while j>2:
        if df.iloc[i][j] is np.nan:
            j=j-1
        else:
            li[i]=df.iloc[i][j]
            year[i]=df.columns[j]
            break

In [ ]:
df['value']=li
df['year']=year
df.head(1)

Seleziono dunque le colonne di interesse per formare due dataset. Uno in formato long contenente anche l'anno, l'altro in formato wide. Quest'ultimo verrà utilizzato per l'integrazione selezionando solo gli indicatori ritenuti più utili alla formazione del db finale.

In [ ]:
soc_eco=df[['Country Name','Country Code', 'Series Name','value', 'year']]
soc_eco.head()

In [ ]:
soc_eco['Country Name'].unique()

In [ ]:
country_dict2={'Argentina':'Argentina',
 'Australia':'Australia',
 'Austria':'Austria',
 'Belgium':'Belgium',
 'Brazil':'Brazil',
 'Canada':'Canada',
 'Chile':'Chile',
 'Colombia':'Colombia',
 'Denmark':'Denmark',
 'Finland':'Finland',
 'France':'France',
 'Germany':'Germany',
 'Greece':'Greece',
 'Iceland':'Iceland',
 'India':'India',
 'Indonesia':'Indonesia',
 'Ireland':'Ireland',
 'Italy':'Italy',
 'Japan':'Japan',
 'Kenya':'Kenya',
 'Luxembourg':'Luxembourg',
 'Mexico':'Mexico',
 'Netherlands':'Netherlands',
 'New Zealand':'New Zealand',
 'Norway':'Norway',
 'Pakistan':'Pakistan',
 'Poland':'Poland',
 'Portugal':'Portugal',
 'South Africa':'South Africa',
 'Spain':'Spain',
 'Sweden':'Sweden',
 'Switzerland':'Switzerland',
 'Thailand':'Thailand',
 'Uganda':'Uganda',
 'United Kingdom':'United Kingdom',
 'United States':'United States'}

In [ ]:
soc_eco['Country Name']=soc_eco['Country Name'].map(country_dict2)

In [ ]:
soc_eco['Country Name'].isna().sum()

In [ ]:
wide_format=soc_eco.pivot(index=['Country Name','Country Code'], columns='Series Name', values='value')
wide_format.head()

In [ ]:
wide_format = wide_format.iloc[:, [4,10,11,14,15,16,17,18,19,20,21,24,25,26,28,29,36,37,38,41]]
wide_format.head()

In [ ]:
df_soceco=wide_format.copy()

In [ ]:
df_soceco.reset_index(inplace=True)
df_soceco.head()

In [ ]:
df_soceco.rename(columns={'Country Name':'country', 'Country Code':'iso'}, inplace=True)
df_soceco.head()

#Importazione e pulizia dataset indicatori ambientali reperibili sul sito [EPI website](https://epi.yale.edu/downloads) alla voce EPI results. 




In [ ]:
df2=pd.read_csv('epi2020results20200604.csv')
df2.head()

Seleziono solo le colonne relative agli indici scelti.

In [ ]:
dfindices=df2.loc[:, df2.columns.str.contains('country|iso|EPI|HLT|ECO|AIR|H2O|HMT|WMG|BDH|ECS|FSH|CCH|APE|AGR|WRS')]
dfindices.head()

In [ ]:
dfindices=dfindices.loc[:, ~dfindices.columns.str.contains('rnk.new')]

In [ ]:
dfindices['country'].unique()

In [ ]:
df_epi=dfindices.copy()

**Integrazione dei due dataset.**

Importo un nuovo df contente l'iso e la regione di appartenenza di ogni paese al fine di integrarlo con il df relativo agli indicatori EPI.

In [ ]:
tab = pd.read_csv('epi2020countryattributes20200604.csv')

In [ ]:
tab = tab[['country', 'region']]

In [ ]:
df_merged = pd.merge(df_epi, tab, left_on='country', right_on='country')
df_merged

Per mezzo della colonna iso unisco il risultante del join precedente al df relativo ai dati socio economici.
Le colonne vengono rinominate per facilitare un futuro accesso.


In [ ]:
final = pd.merge(df_soceco, df_merged, on='iso', how='left')
final.head()

In [ ]:
final.drop('country_y', axis=1, inplace=True)
final.head(1)

In [ ]:
len(final['iso'])

In [ ]:
diz = {'country_x': "Country",
       'iso': "Iso", 
       'Alternative and nuclear energy (% of total energy use)': "NUC_energy",
       "Educational attainment, at least Bachelor's or equivalent, population 25+, total (%) (cumulative)": "EDU_bachelor", 
       'Educational attainment, at least completed post-secondary, population 25+, total (%) (cumulative)': "EDU_secondary",
       'Electricity production from coal sources (% of total)': "ELE_coal",
       'Electricity production from hydroelectric sources (% of total)': "ELE_hydr",
       'Electricity production from natural gas sources (% of total)': "ELE_gas",
       'Electricity production from nuclear sources (% of total)': "ELE_nuc",
       'Electricity production from oil sources (% of total)': "ELE_oil",
       'Employment in agriculture (% of total employment) (modeled ILO estimate)': "EMP_prim",
       'Employment in industry (% of total employment) (modeled ILO estimate)': "EMP_seco",
       'Employment in services (% of total employment) (modeled ILO estimate)': "EMP_terz",
       'GDP per capita (current US$)': "GDP",
       'GDP per capita growth (annual %)': "GDP_growth",
       'Gini index (World Bank estimate)': "Gini",
       'Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)': "Labor_force",
       'Life expectancy at birth, total (years)': "Life_exp",
       'Population, total': "Pop",
       'Poverty gap at $1.90 a day (2011 PPP) (%)': "Poverty_gap",
       'Renewable electricity output (% of total electricity output)': "ELE_renew",
       'Unemployment, total (% of total labor force) (modeled ILO estimate)': "Unemployment",
       'region': "Region"
       }

In [ ]:
final = final.rename(columns=diz) 
final.columns

In [ ]:
final.head()

Il dataset che segue è stato realizzato ad hoc raccogliendo i dati presso i siti [Datareportal](https://datareportal.com) e da [Statcounter](https://gs.statcounter.com).

In [ ]:
social = pd.read_excel('/content/drive/MyDrive/DM&DV Project/Dati/Indicatori/dati social.xlsx')
social.head()

In [ ]:
social = social.iloc[0:36]
social

In [ ]:
diz=data.groupby('Country').count()['testo'].to_dict()
social['numero tweet']=social['Country'].map(diz)

In [ ]:
social['twitter_pop_mln']=social.apply(lambda x: (x['social media users (mln)']*x['% utenti twitter'])/100, axis=1)

In [ ]:
final['prop_tweet']=social.apply(lambda x: x['numero tweet']/(x['twitter_pop_mln']) ,axis=1)

In [ ]:
ddf={}
for i in np.unique(data['Country']):
    dtweet = data[data['Country']==i]
    ddf[i]=len(np.unique(dtweet['username']))

In [ ]:
ddf

In [ ]:
social['username_count']=social['Country'].map(ddf)

In [ ]:
social.head()

In [ ]:
final['prop_user'] = social.apply(lambda x: x['username_count']/(x['twitter_pop_mln']) ,axis=1)

In [ ]:
final.head()

In [ ]:
dataTweet.head()


In [ ]:
naz_prov = pd.merge(final, result, left_on='Country', right_on='country')
naz_prov.head()

In [ ]:
naz_prov['Env_Score'] = naz_prov.apply(lambda x: {'EPI':x['EPI.new'],'HLT':x['HLT.new'],'ECO':x['ECO.new']}, axis=1)

naz_prov['Env'] = naz_prov.apply(lambda x: {'NUC_energy':x['NUC_energy'],'ELE_coal':x['ELE_coal'],
                                    'ELE_hydr':x['ELE_hydr'],'ELE_gas':x['ELE_gas'],
                                    'ELE_nuc':x['ELE_nuc'],'ELE_oil':x['ELE_oil'],
                                    'ELE_renew':x['ELE_renew'],'AIR':x['AIR.new'],
                                    'H2O':x['H2O.new'],'HMT':x['HMT.new'],
                                    'WMG':x['WMG.new'],'BDH':x['BDH.new'],
                                    'ECS':x['ECS.new'],'FSH':x['FSH.new'],
                                    'CCH':x['CCH.new'],'APE':x['APE.new'],'AGR':x['AGR.new'],
                                    'WRS':x['WRS.new'],'AIR.grw':x['AIR.change'],
                                    'H2O.grw':x['H2O.change'],'HMT.grw':x['HMT.change'],
                                    'WMG.grw':x['WMG.change'],'BDH.grw':x['BDH.change'],
                                    'ECS.grw':x['ECS.change'],'FSH.grw':x['FSH.change'],
                                    'CCH.grw':x['CCH.change'],'APE.grw':x['APE.change'],
                                    'AGR.grw':x['AGR.change'],'WRS.grw':x['WRS.change']}, axis=1)

naz_prov['SocEco'] = naz_prov.apply(lambda x: {'GDP':x['GDP'],'GDP_grw':x['GDP_growth'],'Gini':x['Gini'],
                                       'Unemployment':x['Unemployment'],'Labor_force':x['Labor_force'],
                                       'EMP_prim':x['EMP_prim'],'EMP_seco':x['EMP_seco'],
                                       'EMP_terz':x['EMP_terz'],'Poverty_gap':x['Poverty_gap'],
                                       'EDU_bachelor':x['EDU_bachelor'],'EDU_secondary':x['EDU_secondary'],
                                       'Life_exp':x['Life_exp'],'Pop':x['Pop']}, axis=1)

naz_prov['Sentiment'] = naz_prov.apply(lambda x: {'polarity':x['polarity'], 'polarity_std':x['polarity_std']}, axis=1)
naz_prov['TweetProportions'] = naz_prov.apply(lambda x: {'prop_tweet':x['prop_tweet'], 'prop_user':x['prop_user']}, axis=1)


In [ ]:
naz_prov.head()

In [ ]:
country_tweets=[]
country_words=[]
country_bi=[]
country_tri=[]
for paese in pd.unique(naz_prov["Country"]):
    
    tweets=[]
    subset=dataTweet[dataTweet["country"]==paese]

    for tw in subset.itertuples():
        single_tw={
            "Originale":tw.originale,
            "Testo":tw.testo,
            "Lingua":tw.lingua,
            "Polarity":tw.polarity,
            "Subjectivity":tw.subjectivity
            }

        tweets.append(single_tw)
    country_tweets.append(tweets)
    
    parole={}

    for parola in dfwordcloud[dfwordcloud["country"]==paese].itertuples():
        parole[parola.words]=parola.freq
    country_words.append(parole)
    
    bi={}
    tri={}
    for element in df_n_gram[df_n_gram["country"]==paese].itertuples():
        bi[element.bigram]=element.freq_2
        tri[element.trigram]=element.freq_3
    country_bi.append(bi)
    country_tri.append(tri)

In [ ]:
naz_prov['Tweets'] = country_tweets
naz_prov['MostFrequentWords'] = country_words
naz_prov['Bigrams'] = country_bi
naz_prov['Trigrams'] = country_tri

In [ ]:
docs_list =[]
  
# Iterate over each row
for index, rows in naz_prov.iterrows():
    document ={'Country':rows.Country,
               'Env_score':rows.Env_Score,
               'Env':rows.Env, 'SocEco':rows.SocEco,
               'Sentiment':rows.Sentiment,
               'TweetProportions':rows.TweetProportions,
               'Tweets':rows.Tweets,
               'MostFrequentWords':rows.MostFrequentWords,
               'Bigrams':rows.Bigrams,
               'Trigrams':rows.Trigrams}
    docs_list.append(document)

In [ ]:
docs_list[0].items()

In [ ]:
import pymongo
from pymongo import dnspython
from pymongo import MongoClient

In [ ]:
username='username'
password='password'

In [ ]:
client = pymongo.MongoClient("mongodb+srv://{}:{}@cluster0.2d0cl.mongodb.net/cluster0?retryWrites=true&w=majority".format(username, password))

In [ ]:
db = client.progettoDMDV

In [ ]:
nations = db.nations

In [ ]:
nations.insert_many(docs_list)

In [ ]:
client.close()